# Projektna naloga

Za projektno nalogo sem se odločila, da bom analizirala podatke o rakavih obolenjih. Glavni cilj te projektne naloge je, da temeljito analizira statistične podatke posameznega tipa raka ter te podatke ustrezno in razumljivo predstavi.

Najprej uvozimo knjižnjice, ki jih bomo potrebovali. 

In [15]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os

Za tem definiramo spletno stran iz katere jemljemo podatke in jo z BeautifulSoup uvozimo v spremenljivko `bs_spletna_stran`.

In [16]:
link = 'https://seer.cancer.gov'
link_podrocja = link + '/statfacts/'

spletna_stran = urlopen(link_podrocja).read().decode('UTF-8')
bs_spletna_stran = BeautifulSoup(spletna_stran, 'html.parser')

Na spletni strani pogledamo kje se nahajajo podatki, ki jih želimo. Ugotovimo da so zajeti v razredu `alphaList`. Z metodo `.find()` poiščemo vse podatke v `alphaList` in jih shranimo v spremenljivko seznam_podrocij.

In [18]:
seznam_podrocij = bs_spletna_stran.find('div', attrs={'class': 'alphaList'})

Za boljši pregled ustvarimo prazen slovar `slovar_podrocij` v katerega bomo shranili kot ključ tip raka in pod vrednosti povezave za vsak tip posebaj.

In [19]:
slovar_podrocij = {}

Na spletni strani so nekateri podatki neuporabni in jih zato lahko izpustimo. 

In [20]:
nepotrebne_teme = [
    'Female Breast Subtypes', 'Chronic Lymphocytic Leukemia/Small Lymphocytic Lymphoma (CLL/SLL)',
    'Diffuse Large B-Cell Lymphoma (DLBCL)', 'Follicular Lymphoma', 'Lip'
]

for podrocje in seznam_podrocij.find_all('a'):
    if podrocje.get_text() in nepotrebne_teme:
        continue
    slovar_podrocij[podrocje.get_text()] = podrocje.get('href')

Za hitrejše in bolj pregledno kodo, bomo napisali funkcijo, ki bom iz spletne strani pobrala potrebne podatke. Ker so podatki shranjeni pod različne značke, bomo znotraj funkcije napisali pomožne funkcije. V vsaki izmed teh funkcij najprej po znački poiščemo podatke in jih dobimo z `.get_text()`. Vse dobljene podatke shranimo v nov prazen slovar `podatki`, kjer ključe določimo z tipom podatka, za vrednosti pa uzamemo poiskani podatek.

In [22]:
def pridobi_podatke(povezava):
    podatki = {}

    def novi_in_smrtni_primeri(vrsta, podatek):
        for p in podatek.find_all('p'):
            naslov = p.find('span').get_text()
            vrednost = p.find('span').find_next().get_text().replace(',', '')
            estimated = re.search(r'^Estimated', naslov)

            if estimated:
                podatki['prb_' + vrsta] = vrednost
            else:
                podatki['delez_' + vrsta] = vrednost

    def ostali_podatki(kljuc, podatek):
        tag = 'td'
        if 'povp_starost' in kljuc:
            tag = 'strong'
        podatek_spol = podatek.find(tag).get_text().replace(',', '')
        podatki[kljuc] = podatek_spol

    def spletna_povezava(oznaka, tip, sez):
        podatki_za_predelavo = spletna_podrocje.find(oznaka, attrs={tip: sez})
        return podatki_za_predelavo

    link_tema = link + povezava
    spletna_podrocje = BeautifulSoup(urlopen(link_tema).read().decode('UTF-8'), 'html.parser')

    # novi primeri
    if spletna_povezava('div', 'class', ['new', 'glanceBox']):
        novi_in_smrtni_primeri('novi', spletna_povezava('div', 'class', ['new', 'glanceBox']))

    # smrtni primeri
    if spletna_povezava('div', 'class', ['death']):
        novi_in_smrtni_primeri('smrtni', spletna_povezava('div', 'class', ['death']))

    # podatki o relativnem prezivetje
    if spletna_povezava('div', 'class', ['col-lg-4', 'offset-lg-1']):
        for div in spletna_povezava('div', 'class', ['col-lg-4', 'offset-lg-1']).find_all('div'):
            podatki_prezivetje = div.find('strong').get_text()
            podatki['5_letno_realtivno_prezivetje_stevilo'] = podatki_prezivetje

    # podatki o novih primerih moski
    if spletna_povezava('table', 'id', ['scrapeTable_04']):
        ostali_podatki('vse_rase_moski', spletna_povezava('table', 'id', ['scrapeTable_04']))

    # podatki o novih primerih zenske
    if spletna_povezava('table', 'id', ['scrapeTable_05']):
        ostali_podatki('vse_rase_zenske', spletna_povezava('table', 'id', ['scrapeTable_05']))

    # podatki o povprečni starosti za nove primere
    if spletna_povezava('div', 'class', ['statSurv']):
        ostali_podatki('povp_starost', spletna_povezava('div', 'class', ['statSurv']))

    # podatki o smrtnih primerih moski
    if spletna_povezava('table', 'id', ['scrapeTable_07']):
        ostali_podatki('smrtni_vse_rase_moski', spletna_povezava('table', 'id', ['scrapeTable_07']))

    # podatki o smrtnih primerih zenske
    if spletna_povezava('table', 'id', ['scrapeTable_08']):
        ostali_podatki('smrtni_vse_rase_zenske', spletna_povezava('table', 'id', ['scrapeTable_08']))

    # podatki o povprecni starosti smrti
    if spletna_povezava('div', 'class', ['col-lg-4', 'offset-lg-1']):
        ostali_podatki('povp_starost_smrti', spletna_povezava('div', 'class', ['statDie']))

    return podatki

Za boljšo preglednost, podatke v slovarju `podatki` spravimo v prvotni slovar, kjer so ključi tipi raka. Tako imamo v slovarju za vsak tip raka posebaj zbrane vse podatke.

In [23]:
for k, v in slovar_podrocij.items():
    slovar_podrocij[k] = pridobi_podatke(v)

Zdaj ustvarimo mapo, kamor bomo shranjevali izrisane grafe. To nam omogoča boljšo preglednost.

In [26]:
mapa_grafi = 'grafi'
mapa_obstaja = os.path.exists(mapa_grafi)
if not mapa_obstaja:
    os.makedirs(mapa_grafi)
mapa_grafi += '/'

Ker se iz slovarja zelo težko znajdemo bomo raje naredili csv datoteko, kjer bodo podatki bolj razvidni. Zopet bomo definiral funkcijo in sicer `ustvari_csv`, kjer bomo najprej odpli datoteko, nato ustvarimo glavo csv datoteko, to so imena stolpcev naše tabele. Za tem bomo z zanko `for` šli čez vse ključe in vrednosti v slovarju. Tu si pomagamo z metodo `.get()` z katero dobimo željeno vrednost iz slovarja. 

In [28]:
csv_ime_datoteke = 'output.csv'

def ustvari_csv(ime_datoteke):
    with open(ime_datoteke, mode='w', newline='') as file:
        writer = csv.writer(file)

        header = ['Type Of Cancer', 'Estimated New Cases in 2023', '(%) of All New Cancer Cases',
                  'Estimated Deaths in 2023',
                  '(%) of All Cancer Deaths', '5-Year Relative Survival ',
                  'New Cases Male', 'New Cases Female', 'Median Age At Diagnosis',
                  'Death Cases Male', 'Death Cases Female', 'Median Age At Death']
        writer.writerow(header)

        for name, values in slovar_podrocij.items():
            prb_novi = values.get('prb_novi')
            delez_novi = values.get('delez_novi')
            prb_smrti = values.get('prb_smrtni')
            delez_smrti = values.get('delez_smrtni')
            relativno_prezivetje_stevilo = values.get('5_letno_realtivno_prezivetje_stevilo')
            vse_rase_m = values.get('vse_rase_moski')
            vse_rase_z = values.get('vse_rase_zenske')
            povp_starost = values.get('povp_starost')
            smrtni_vse_rase_m = values.get('smrtni_vse_rase_moski')
            smrtni_vse_rase_z = values.get('smrtni_vse_rase_zenske')
            povp_starost_smrti = values.get('povp_starost_smrti')

            row = [name, prb_novi, delez_novi, prb_smrti, delez_smrti, relativno_prezivetje_stevilo,
                   vse_rase_m, vse_rase_z, povp_starost, smrtni_vse_rase_m,
                   smrtni_vse_rase_z, povp_starost_smrti]
            writer.writerow(row)

Zadnja stvar, ki jo moramo narediti je ustvariti csv datoteko. To naredimo tako, da poženemo našo funkcijo `ustvari_csv`.

In [31]:
ustvari_csv(csv_ime_datoteke)

Za anlizo podatkov uporabimo knjižnjico `pandas`. Z `pd.read_csv()` preberemo ustvarjeno csv datoteko. Opazimo, da nekateri podatki mankajo oziroma imajo podatek v obliki teksta. Za lažjo obdelavo podatkov te nadomestili z vrednostjo 0 in zamenjamo tip podatka v numeričnega.

In [65]:
def predelaj_csv_podatke(ime_datoteke, zapolni_privzete=False):
    predelani_podatki = pd.read_csv(ime_datoteke, encoding='cp1252', index_col='Type Of Cancer')
    if zapolni_privzete:
        predelani_podatki = predelani_podatki.fillna(0)
        predelani_podatki.replace('Sex-specific cancer type', 0, inplace=True)

    # številčne podatke spremenimo v numerične za lažjo obdelavo
    stolpci = ['Estimated New Cases in 2023', 'Estimated Deaths in 2023']
    predelani_podatki[stolpci] = predelani_podatki[stolpci].applymap(pd.to_numeric, errors='coerce').astype(int)

    return predelani_podatki

Uporabimo funkcijo in shranimo pod spremenljivko.

In [66]:
vsi_podatki = predelaj_csv_podatke(csv_ime_datoteke, zapolni_privzete=True)

Zdaj ko so podatki v ustrezni obliki jih lahko filtriramo. Najprej bomo prikazali pričakovano število novih primerov. Te uredimo po velikosti z `.sort_values()`.

In [67]:
pricakovani_novi_primeri = vsi_podatki[['Estimated New Cases in 2023']].sort_values('Estimated New Cases in 2023',
                                                                               ascending=False)

Odstranimo vse vrstice, kjer je 0.

In [68]:
pricakovani_novi_primeri = pricakovani_novi_primeri[pricakovani_novi_primeri['Estimated New Cases in 2023'] != 0]

V tabeli so tako prikazani podatki za pričakovano število novih primerov za vsak tip raka posebaj. Ker smo vrednosti uredili po velikosti, lahko iz tabele preberemo kateri tip raka je najpogostejši torej rak dojk.

In [69]:
print(pricakovani_novi_primeri)

                                    Estimated New Cases in 2023
Type Of Cancer                                                 
Breast (Female)                                          297790
Prostate                                                 288300
Lung and Bronchus                                        238340
Colon and Rectum                                         153020
Melanoma of the Skin                                      97610
Bladder                                                   82290
Kidney and Renal Pelvis                                   81800
Non-Hodgkin Lymphoma                                      80550
Uterus                                                    66200
Pancreas                                                  64050
Leukemia                                                  59610
Oral Cavity and Pharynx                                   54540
Thyroid                                                   43720
Liver and Intrahepatic Bile Duct        

Enako naredimo še za smrtne primere.

In [70]:
pricakovani_smrtni_primeri = vsi_podatki[['Estimated Deaths in 2023']].sort_values('Estimated Deaths in 2023',
                                                                              ascending=False)
pricakovani_smrtni_primeri = pricakovani_smrtni_primeri[pricakovani_smrtni_primeri['Estimated Deaths in 2023'] != 0]

Naslednja tabela je zelo podobna prejšnji, le da so tu prikazani podatki za smrtne primere. 

In [71]:
print(pricakovani_smrtni_primeri)

                                    Estimated Deaths in 2023
Type Of Cancer                                              
Lung and Bronchus                                     127070
Colon and Rectum                                       52550
Pancreas                                               50550
Breast (Female)                                        43170
Prostate                                               34700
Liver and Intrahepatic Bile Duct                       29380
Leukemia                                               23710
Non-Hodgkin Lymphoma                                   20180
Brain and Other Nervous System                         18990
Bladder                                                16710
Esophagus                                              16120
Kidney and Renal Pelvis                                14890
Ovary                                                  13270
Uterus                                                 13030
Myeloma                 

Poleg podatkov o novih in smrtnih primerih, je dobro vedeti pri katerem spolu je določen tip pogostejši. V naslednji tabeli so prikazani podatki o odkritih novih primerih razdeljeni glede na moške in ženske. V zadnjem stolpcu pa so podatki o povprečni starosti pri kateri odkrijejo specifičen tip raka. Ponekod se pojavi `\`, to zato ker so določeni tipi specifični za določen spol recimo rak materničnega vrata.

In [45]:
kdo_dobi_tega_raka = vsi_podatki[['New Cases Male', 'New Cases Female', 'Median Age At Diagnosis']].replace(0, '/')
print(kdo_dobi_tega_raka)

                                                   New Cases Male  \
Type Of Cancer                                                      
Anus                                                          1.6   
Bladder                                                      31.7   
Bone and Joint                                                1.2   
Brain and Other Nervous System                                7.4   
Breast (Female)                                                 /   
Cervix Uteri                                                    /   
Childhood Brain and Other Nervous System (Ages ...            3.2   
Childhood Leukemia (Ages 0–19)                                5.4   
Colon and Rectum                                             42.1   
Esophagus                                                     7.2   
Kidney and Renal Pelvis                                      23.5   
Larynx                                                        4.7   
Liver and Intrahepatic Bile Duct  

Seveda nas poleg odkritih novih primerov zanima še kako je z smrtnostjo, zato enako naredimo še za smrtne primere. V tabeli so tako prikazani podatki za vsak spol, kot tudi povprečna starost smrti.

In [46]:
kdo_umre_od_tega_raka = vsi_podatki[['Death Cases Male', 'Death Cases Female', 'Median Age At Death']].replace(0, '/')
print(kdo_umre_od_tega_raka)

                                                   Death Cases Male  \
Type Of Cancer                                                        
Anus                                                            0.3   
Bladder                                                         7.1   
Bone and Joint                                                  0.6   
Brain and Other Nervous System                                  5.4   
Breast (Female)                                                   /   
Cervix Uteri                                                      /   
Childhood Brain and Other Nervous System (Ages ...              0.7   
Childhood Leukemia (Ages 0–19)                                  0.6   
Colon and Rectum                                               15.7   
Esophagus                                                       6.7   
Kidney and Renal Pelvis                                         5.1   
Larynx                                                          1.6   
Liver 

V spodaj prikazani tabeli so podatki o 5 letni stopnji preživetja.

In [47]:
moznost_prezivetja = vsi_podatki[['5-Year Relative Survival ']].replace(0, '/')
print(moznost_prezivetja)

                                                   5-Year Relative Survival 
Type Of Cancer                                                              
Anus                                                                   70.4%
Bladder                                                                77.9%
Bone and Joint                                                         68.9%
Brain and Other Nervous System                                         33.8%
Breast (Female)                                                        90.8%
Cervix Uteri                                                           67.2%
Childhood Brain and Other Nervous System (Ages ...                         /
Childhood Leukemia (Ages 0–19)                                             /
Colon and Rectum                                                       65.0%
Esophagus                                                              21.7%
Kidney and Renal Pelvis                                                77.6%

Podatke želimo predstaviti tudi grafično, zato definiramo gunkcijo za generiranje grafov.

In [48]:
def generiraj_graf(naslov, vrednosti):
    ax1 = vrednosti.plot(kind='barh', legend=False, figsize=(25, 8), width=0.8, color='#4169E1')
    ax1.set_title(naslov)
    ax1.set_ylabel(' ')
    plt.savefig(mapa_grafi + naslov + '.jpg', format='jpeg')
    plt.close()

Prvi graf prikazuje pričakovane nove primere v letu 2023.

In [50]:
generiraj_graf('Estimated New Cases in 2023', pricakovani_novi_primeri)

Drugi graf prikazuje pričakovane smrtne primere v letu 2023.

In [51]:
generiraj_graf('Estimated Deaths in 2023', pricakovani_smrtni_primeri)

Enako naredimo še za generiranje grafov, kjer bo prikazana primerjava med šteilom moških in številom žensk, ki dobijo oziroma umrejo za rakom. Za vsak tip raka se bo posebaj izrisal graf, ki bo prikaz delež moških in delež žensk pri katerem so odkrili raka oziroma so umrli za rakom.

In [59]:
def generiraj_graf_primerjave(v1, v2, naslov):
    primerjava = vsi_podatki[[v1, v2]]
    primerjava.loc[:, [v1, v2]] = primerjava[[v1, v2]].applymap(pd.to_numeric, errors='coerce')

    for index, row in primerjava.iterrows():
        values = row.values

        plt.bar(range(len(values)), values, color=['#A4D3EE', '#FFAEB9'])

        plt.ylim(bottom=0)
        plt.title(f'Type of cancer: {index}')

        plt.savefig(f'{mapa_grafi}{naslov}{index}.jpg', format='jpeg')
        plt.clf()

Generiramo grafe tako, da uporabimo funkcijo.

In [60]:
generiraj_graf_primerjave('New Cases Male', 'New Cases Female', 'bar_plot_new_cases_')
generiraj_graf_primerjave('Death Cases Male', 'Death Cases Female', 'bar_plot_death_cases_')

<Figure size 640x480 with 0 Axes>

Na koncu bomo predstavili še kakšen delež predstavlja določen tip raka v novih primerih in pa v smrtnih primerih. Prvo kar moramo narediti je filtrirati podatke, saj niso vsi uporabni.

In [55]:
procenti = vsi_podatki[['(%) of All New Cancer Cases', '(%) of All Cancer Deaths']]
procenti = procenti[procenti['(%) of All New Cancer Cases'] != 0]

procenti.loc[:, '(%) of All New Cancer Cases'] = procenti['(%) of All New Cancer Cases'].str.replace('%', '')
procenti.loc[:, '(%) of All Cancer Deaths'] = procenti['(%) of All Cancer Deaths'].str.replace('%', '')
procenti.loc[:, ['(%) of All New Cancer Cases', '(%) of All Cancer Deaths']] = procenti[
    ['(%) of All New Cancer Cases', '(%) of All Cancer Deaths']].applymap(pd.to_numeric, errors='coerce')

procenti_novi = procenti[['(%) of All New Cancer Cases']]
procenti_smrtni = procenti[['(%) of All Cancer Deaths']]

Ker smo že napisali funkcijo za generiranje grafov, jo lahko tu samo pokličemo in vanjo ustavimo ustrezne podatke.

In [58]:
generiraj_graf('(%) of All New Cancer Cases', procenti_novi)
generiraj_graf('(%) of All Cancer Deaths', procenti_smrtni)